In [69]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import os

%matplotlib inline

path = '/'.join(os.getcwd().split('/')[:-1])
print path

/Volumes/Dropbox-1/largetransfer/luc/carter


In [70]:
tci = pd.read_csv(path+'/data/model_data/tci_2_1.csv')
ppns = set(tci['ppn'])
ppns_num = set(tci['ppn'].apply(lambda x: x.replace('-','')))

## Violations

In [71]:
vs = pd.read_csv(path+'/data/clean_data/violations.csv', parse_dates = [1,4])

In [72]:
vs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129318 entries, 0 to 129317
Data columns (total 18 columns):
VIN                     129318 non-null object
v_file_date             129318 non-null datetime64[ns]
v_wf_task               129318 non-null object
v_wf_task_status        128371 non-null object
v_wf_task_date          129318 non-null datetime64[ns]
v_type_of_violation     123474 non-null object
parcel1                 56581 non-null object
parcel                  129318 non-null object
cond_gar                14422 non-null float64
cond                    32184 non-null float64
other                   22020 non-null float64
maint                   50331 non-null float64
fire                    3371 non-null float64
int_ext                 1123 non-null float64
vin_id                  96775 non-null object
violation_issue_date    96775 non-null object
vn_created              96775 non-null float64
vn_source               96775 non-null float64
dtypes: datetime64[ns](2), float

In [73]:
set(vs.v_type_of_violation)

{nan,
 '14 Day Condemnation Garage',
 '14 Day Condemnation MS',
 '14 Day Vacate',
 '24 Hour Vacate',
 '30 Day Condemnation',
 '30 Day Condemnation Garage',
 '30 Day Condemnation MS',
 '30 Day Damage Repair Shed',
 '30 Day Det. Cond. Garage',
 '30 Day Det. Cond. MS',
 '30 Day Deteriorated Garage',
 '30 Day Fire Condemnation Gara',
 '30 Day Fire Condemnation MS',
 '30 Day Fire Condemnation Shed',
 '30 Day Fire Damage Repair Gar',
 '30 Day Fire Damage Repair MS',
 '30 Day Vacate',
 '7 Day Condemnation Garage',
 '7 Day Condemnation MS',
 '7 Day Vacate',
 '72 HR Emergency Cond. Garage',
 '72 HR Emergency Cond. MS',
 '72 HR Emergency Fire Cond. Ga',
 '72 HR Emergency Fire Cond. MS',
 'Appurtenant Structure',
 'Auto Repairing',
 'C of O Inspection',
 'Change of Use',
 'Commercial Maintenance',
 'Condemnation',
 'Condemnation - Garage',
 'Condemnation - Main Structure',
 'Condemnation - Shed',
 'Display Signs',
 'Electrical',
 'Elevator',
 'Erect Without a Permit',
 'Exit Traffic',
 'Exterior 

### Violation Task

In [74]:
vs_task = pd.DataFrame(vs.groupby('v_wf_task').count()['VIN'])
vs_task.columns = ['violations_tasks']
vs_task[vs_task.violations_tasks > 1]

,violations_tasks
v_wf_task,
Application Acceptance,36923
Closure,24403
Condemnation,2225
Demolition Approval,211
Inspection,56556
Non-Condemnation,4347
Prosecution,758
Search Warrant,3895


In [75]:
for task in set(vs.v_wf_task):
    tci = pd.merge(tci, vs.loc[vs.v_wf_task==task,['parcel','v_wf_task']].groupby('parcel').count(), how='left', left_on = 'ppn', right_index=True)
    tci = tci.rename(columns={'v_wf_task': task.lower()+' count'})
    tci[task.lower()+' count'] = tci[task.lower()+' count'].fillna(0)
#     print task, len(a), sum(a.vacant)*1.0/len(a.vacant)

In [76]:
# def check_task(x):
#     if x in parcel_set:
#         return 1
#     else:
#         return 0
    
# for task in vs_task.index:
#     parcel_set = set(vs.loc[vs.v_wf_task==task,'parcel'])
#     tci['task_'+task[0:3].lower()] = tci['ppn'].apply(check_task)

### Violations count 

In [77]:
tci.columns

Index([u'Parcel ID', u'ppn', u'vacant', u'Date', u'Survey Category',
       u'Survey Result', u'zip', u'pclass', u'totusabl', u'tmktval',
       u'condition', u'parcel_x', u'condition_value', u'parcel_y',
       u'style_filtered', u'parcel_x.1', u'ownerocc_value', u'parcel_y.1',
       u'totbldgs', u'parcel_x.2', u'yrbuilt_filtered', u'parcel_y.2',
       u'rextwall', u'parcel', u'vacant_block', u'owner', u'renter',
       u'median_rent', u'property_crimes', u'burglaries', u'other_drug',
       u'part_one_crimes', u'part_two_crimes', u'bachelors+', u'poverty_rate',
       u'median_hh_income', u'median_fam_income', u'white', u'black', u'asian',
       u'other', u'hispanic', u'young', u'middle', u'old', u'num_of_sales',
       u'quit_claim_deed', u'warranty_deed', u'limited_warranty',
       u'survivorship_deed', u'fiduciary_deed', u'AMOUNT_PAID', u'SALE_VALID',
       u'num_of_transfers', u'DEED_TYPE', u'county_land_bank', u'vindall_Y',
       u'vindall_P', u'closure count', u'applicati

In [78]:
vs_count = vs.groupby('parcel').count()
vs_count = vs_count.rename(columns={'VIN':'violations_number'})
tci = pd.merge(tci, vs_count[['violations_number']], \
               how='left', left_on='ppn', right_index=True)
tci.loc[tci.violations_number.isnull(), 'violations_number'] = 0

In [79]:
vs_count = vs[vs['v_file_date']>dt.datetime(2014,3,1)].groupby('parcel').count()
vs_count = vs_count.rename(columns={'VIN':'violations_number_1_yr'})
tci = pd.merge(tci, vs_count[['violations_number_1_yr']], \
               how='left', left_on='ppn', right_index=True)
tci.loc[tci.violations_number.isnull(), 'violations_number_1_yr'] = 0

### Violation types

In [80]:
vs_type = vs[['v_type_of_violation','parcel']].groupby('v_type_of_violation').count()
vs_type = vs_type.rename(columns={'parcel':'violations_type'})

In [81]:
violation_types = vs_type[vs_type['violations_type']>500]

In [82]:
violation_types

,violations_type
v_type_of_violation,
30 Day Condemnation,852
30 Day Condemnation Garage,993
30 Day Condemnation MS,1170
30 Day Fire Damage Repair MS,728
Condemnation - Garage,11885
Condemnation - Main Structure,27971
Condemnation - Shed,755
Exterior Maintenance,44273
Fire Damage,3293


In [83]:
for vio in violation_types.index:
    tci = pd.merge(tci, vs.loc[vs.v_type_of_violation==vio, ['v_type_of_violation','parcel']].groupby('parcel').count(), left_on='ppn', right_index=True, how='left')
    tci = tci.rename(columns={'v_type_of_violation': 'v_'+vio.lower()+'_count'})
    tci['v_'+vio.lower()+'_count'] = tci['v_'+vio.lower()+'_count'].fillna(0)

## Complaints

In [84]:
cp = pd.read_csv(path+'/data/clean_data/complaints.csv', parse_dates = [2],index_col=[0])

In [85]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232110 entries, 30389 to 370166
Data columns (total 19 columns):
complaint_id           232110 non-null object
c_file_date            232110 non-null datetime64[ns]
c_wf_task              232110 non-null object
c_wf_task_status       224628 non-null object
c_wf_task_date         232109 non-null object
c_type_of_complaint    226786 non-null object
cdc_ri                 65 non-null float64
cdc_corrected          180 non-null float64
cdc_referred           504 non-null float64
othercomp              190166 non-null float64
parcel                 232110 non-null object
parcel1                16837 non-null float64
cdc_comp               0 non-null float64
cdccia_comp            0 non-null float64
council_comp           0 non-null float64
mac_comp               0 non-null float64
public_comp            0 non-null float64
other_comp             0 non-null float64
comp_source            224147 non-null object
dtypes: datetime64[ns](1), float6

In [86]:
cp.head(2)

,complaint_id,c_file_date,c_wf_task,c_wf_task_status,c_wf_task_date,c_type_of_complaint,cdc_ri,cdc_corrected,cdc_referred,othercomp,parcel,parcel1,cdc_comp,cdccia_comp,council_comp,mac_comp,public_comp,other_comp,comp_source
30389,CMP06037794,2006-03-06,Closure,Closed-ADMIN,05/28/2009,NaN,NaN,NaN,NaN,1,002-23-031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30390,CMP06037794,2006-03-06,Inspection,Inspection Denied,05/28/2009,NaN,NaN,NaN,NaN,1,002-23-031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
cp.groupby('c_type_of_complaint').count()['complaint_id']

c_type_of_complaint
Black Mold                                  233
Collapsing Structure                       4275
Complete interior/exterior                24196
Court Ordered Inspection                     92
Daycare                                      12
Debris/Garbage/Junk                         381
Debris/Garbage/Junk (Occupied)             1725
Debris/Garbage/Junk (Vacant)                886
Doors/Gate/Locks                            497
Driveway                                    358
Dumping                                       8
Dumping Private (Vacant)                    175
Dumping Public                               35
Electrical                                 6380
Elevator                                     24
Fence                                      1426
Fire Damage                               14394
Foundation issues (animals entering)         27
Garage/Shed/Outbuildings                   3953
General Exterior Maintenance              22916
Graffiti            

In [88]:
types = ['Black Mold','Collapsing Structure','Complete interior/exterior', \
         'Debris/Garbage/Junk (Occupied)','Electrical','Fire Damage', \
         'General Exterior Maintenance','No Permit','O.V.V.', 'Plumbing']

### Complaint type

In [89]:
cp_type = pd.DataFrame(cp.groupby('c_type_of_complaint').count()['complaint_id'])
cp_type.columns = ['complaint_type']

In [90]:
cp_type[cp_type['complaint_type']/len(ppns)>0.05].complaint_type

c_type_of_complaint
Complete interior/exterior       24196
Electrical                        6380
Fire Damage                      14394
General Exterior Maintenance     22916
No Permit                         6917
O.V.V.                          101862
Name: complaint_type, dtype: int64

In [91]:
for complaint in types:
    t = cp.loc[cp.c_type_of_complaint==complaint,['c_type_of_complaint','parcel']].groupby('parcel').count()
    t = t.rename(columns={'c_type_of_complaint': 'c_'+complaint})
    tci = pd.merge(tci, t, left_on='ppn', right_index=True, how='left')
    tci['c_'+complaint] = tci['c_'+complaint].fillna(0)

### Complaint count (all)

In [92]:
cp_count = pd.DataFrame(cp.groupby('parcel').count()['complaint_id'])
cp_count.columns = ['complaint_count']
tci = pd.merge(tci, cp_count, \
               how='left', left_on='ppn', right_index=True)
tci.loc[tci.complaint_count.isnull(), 'complaint_count'] = 0

In [93]:
sum(tci.loc[tci.complaint_count>5, 'vacant'])*1.0/len(tci.loc[tci.complaint_count>5, 'vacant'])

0.10043602465794617

### Complaint count (6 months)

In [94]:
cp_count = pd.DataFrame(cp[cp.c_file_date>dt.datetime(2013, 9, 1)].groupby('parcel').count()['complaint_id'])
cp_count.columns = ['complaint_count_6_mon']
tci = pd.merge(tci, cp_count, \
               how='left', left_on='ppn', right_index=True)
tci.loc[tci.complaint_count_6_mon.isnull(), 'complaint_count_6_mon'] = 0

In [95]:
sum(tci.loc[tci.complaint_count_6_mon>2, 'vacant'])*1.0/len(tci.loc[tci.complaint_count_6_mon>2, 'vacant'])

0.29947460595446584

### Complain count (3 months)

In [96]:
cp_count = pd.DataFrame(cp[cp.c_file_date>dt.datetime(2013, 12, 1)].groupby('parcel').count()['complaint_id'])
cp_count.columns = ['complaint_count_3_mon']
tci = pd.merge(tci, cp_count, \
               how='left', left_on='ppn', right_index=True)
tci.loc[tci.complaint_count_3_mon.isnull(), 'complaint_count_3_mon'] = 0

In [97]:
sum(tci.loc[tci.complaint_count_3_mon>0, 'vacant'])*1.0/len(tci.loc[tci.complaint_count_3_mon>0, 'vacant']) ,len(tci.loc[tci.complaint_count_3_mon>0, 'vacant'])

(0.31665363565285382, 1279)

In [98]:
tci.to_csv(path+'/data/model_data/tci_2_2.csv', index=False)

In [99]:
tci.shape

(122560, 98)